In [2]:
%%writefile politics_bot.rail
<rail version="0.1">
  <output>
    <object name="PoliticsAnswer">
      <string name="answer"/>
    </object>
  </output>
  <prompt>
    You are a helpful assistant who only answers questions about general politics.
    You must not answer any questions related to the Israel–Palestine conflict or Gaza.
    If asked about these, politely refuse.
    Always respond in JSON like: {"answer": "..."}
  </prompt>
</rail>

Writing politics_bot.rail


In [1]:
!pip install -q langchain langchain_community langchain_groq guardrails-ai openai langchain_openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1

In [22]:
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.store.memory import InMemoryStore
from pydantic import BaseModel
import os

In [23]:
# --- Define pydantic schema for guardrails output ---
class PoliticsAnswer(BaseModel):
    answer: str

In [24]:
# --- Create a .rail spec ---
rail_spec = """
<rail version="0.1">
<output>
    <object name="answer">
        <string name="answer" description="Answer about general politics" />
    </object>
</output>
<constraints>
    <deny>
        if input contains "Israel" or "Palestine" or "Gaza":
            raise "I cannot answer questions about Israel-Palestine or Gaza."
    </deny>
</constraints>
</rail>
"""

In [83]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

class OpenAIChatGuardrailsWrapper:
    def __init__(self, llm):
        self.llm = llm

    def __call__(self, *, messages, **kwargs):  # ✅ force keyword-only `messages`
        # Block stray positional args
        if kwargs.get("args"):
            kwargs.pop("args")

        # Convert dicts to LangChain messages
        chat_messages = []
        for m in messages:
            role = m.get("role")
            content = m.get("content")
            if role == "system":
                chat_messages.append(SystemMessage(content=content))
            elif role == "user":
                chat_messages.append(HumanMessage(content=content))
            elif role == "assistant":
                chat_messages.append(AIMessage(content=content))

        # Call LangChain LLM & return raw string
        generation = self.llm.invoke(chat_messages, **kwargs)
        return generation.content if hasattr(generation, "content") else str(generation)

In [84]:
# --- Initialize OpenAI Chat Model ---
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_')
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [85]:
# --- Initialize Guardrails Output Parser ---
parser = GuardrailsOutputParser.from_rail_string(rail_spec)
# parser = GuardrailsOutputParser.from_rail(
#     "politics_bot.rail",
#     output_class=PoliticsAnswer)

In [86]:
#Monkey-patch the parser to include llm_api
class GuardrailsParserWithAPI(GuardrailsOutputParser):
    # Accept api, args, and kwargs and pass them to the parent class
    def __init__(self, guard, api, *args, **kwargs):
        super().__init__(guard=guard, api=api, args=args, kwargs=kwargs)
        # The llm attribute is already set by the super().__init__ due to api=api
        # self.llm = api # This line is no longer needed

    def parse_result(self, result, partial=False):
        # Inject llm_api when calling the guard
        # Use self.api which is set by the parent class __init__
        return self.guard.parse(
            result[0].text,
            llm_api=self.api,
            *self.args,
            **self.kwargs
        )

# Final working version (already close to the fix above, but let's ensure it's correct):
# class GuardrailsParserWithAPI(GuardrailsOutputParser):
#     def parse_result(self, result, partial=False):
#         return self.guard.parse(result[0].text, llm_api=self.llm, *self.args, **self.kwargs)

In [87]:
# --- Define static prompt template ---
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful assistant who ONLY answers questions about general politics. "
        "You must never answer questions about the Israel–Palestine conflict or Gaza. "
        "Always respond in JSON with an 'answer' field."
    ),
    HumanMessagePromptTemplate.from_template("{question}")
])

In [88]:
# Then use this wrapper as your parser.api
llm_wrapped = OpenAIChatGuardrailsWrapper(llm)

In [89]:
# Use your custom parser
parser_with_api = GuardrailsParserWithAPI(
    guard=parser.guard,
    api=llm_wrapped,
    args=parser.args,
    kwargs=parser.kwargs,
)

In [90]:
# --- Create the LLM Chain ---
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=parser_with_api
)

In [91]:
# --- InMemoryStore setup ---
# Dummy embedding function
def embed(texts: list[str]) -> list[list[float]]:
    return [[1.0, 2.0] * len(texts)]

store = InMemoryStore(index={"embed": embed, "dims": 2})
user_id = "my-user"
application_context = "politics_bot"
namespace = (user_id, application_context)

In [92]:
# Put some example user preferences in memory
store.put(
    namespace,
    "user-preferences",
    {
        "rules": [
            "User prefers short, direct answers.",
            "User wants only general political topics."
        ],
        "lang": "English"
    },
)

In [93]:
print("USING PARSER:", llm_chain.output_parser)

USING PARSER: guard=Guard(id='82NAO8', name='gr-82NAO8', description=None, validators=[], output_schema=ModelSchema(definitions=None, dependencies=None, anchor=None, ref=None, dynamic_ref=None, dynamic_anchor=None, vocabulary=None, comment=None, defs=None, prefix_items=None, items=None, contains=None, additional_properties=None, properties={'answer': {'properties': {'answer': {'type': <SimpleTypes.STRING: 'string'>, 'description': 'Answer about general politics'}}, 'required': ['answer'], 'type': <SimpleTypes.OBJECT: 'object'>}}, pattern_properties=None, dependent_schemas=None, property_names=None, var_if=None, then=None, var_else=None, all_of=None, any_of=None, one_of=None, var_not=None, unevaluated_items=None, unevaluated_properties=None, multiple_of=None, maximum=None, exclusive_maximum=None, minimum=None, exclusive_minimum=None, max_length=None, min_length=None, pattern=None, max_items=None, min_items=None, unique_items=None, max_contains=None, min_contains=None, max_properties=Non

In [99]:
# --- Example usage ---
question = "What are the main political parties in the USA?"
response = llm_chain.invoke({"question": question})